need to have demographic.csv in directory

In [1]:
!pip install selenium

In [2]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import re
import urllib, json
import csv
from urllib.request import Request, urlopen
from IPython.display import clear_output

In [3]:
demo_df = pd.read_csv('demographic.csv')
demo_df.code = demo_df.code.astype('str')
bursa_site = demo_df.iloc[:,2]

In [4]:
CHROME_PATH = 'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'
chrome_options = Options()
chrome_options.add_argument("--headless")
WINDOW_SIZE = "1920,1080"
# chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.add_argument('--disable-gpu')
header = {'User-Agent': 'Mozilla/5.0'}

In [5]:
link = 'https://ws20.bursamalaysia.com/api/v2/stock_price_data?stock_code='
link2 = '&mode=historical&from_date='
date = '20160111'
link3 = '&ws_a='
link4 = '&ws_m='

In [ ]:
df2 = pd.DataFrame()
df2.index.name = 'code'
x=True
while x==True:
    try:
        try:
            doneindex = pd.read_csv('donelist.csv').values
            doneindex = np.ndarray.flatten(w).tolist()
        except:
            demo = pd.DataFrame(list())
            doneindex = pd.read_csv('donelist.csv').values
            doneindex = np.ndarray.flatten(w).tolist()
        for index,entry in enumerate(bursa_site):
            if len(doneindex) > 782:
                clear_output(wait=True)
                print('breaking for completion')
                break
            if index in doneindex:
                clear_output(wait=True)
                print('previously done loop',index)
                continue
            else:
                driver = webdriver.Chrome('chromedriver.exe',options=chrome_options)
                driver.get(entry)
    #             time.sleep(1)
                target = driver.find_element_by_xpath('//div[@id="stockChartContainer"]')
    #             time.sleep(1)
                text = target.get_attribute('outerHTML')
    #             time.sleep(1)
                code = re.findall(r'data-stock-code="(.+?)"',text)
                wsa = re.findall(r'ws-a="(.+?)"',text)
                wsm = re.findall(r'ws-m="(.+?)"',text)
                dt = '20160111'
                outputlink = link+code[0]+link2+dt+link3+wsa[0]+link4+wsm[0]
                req = Request(url=outputlink, headers = header)
                response = urlopen(req).read()
                response2 = response.decode('utf-8').strip()
                json1 = json.loads(response2)
                lst = json1['historical_data']['data']
                date1 = []
                price = []
                vol = []
                for entry1 in lst:
                    date1.append(entry1['date'])
                    price.append(entry1['low'])
                    vol.append(entry1['vol'])
                jdate = date1[:-1]
                price = price[:-1]
                vol = vol[:-1]
                code = json1['historical_data']['data'][0]['id']
                date = []
                for entry2 in jdate:
                    d = time.gmtime(entry2/1000)
                    d = time.strftime('%Y-%m-%d',d)
                    date.append(d)
                date.insert(0,'code')
                price.insert(0,str(code))
                dat1 = pd.DataFrame(data = [price],columns=date)
                df2 = df2.append(dat1)
                driver.close()
                doneindex.append(index)
                clear_output(wait=True)
                finaldat = pd.merge(demo_df,df2,left_on='code',right_on='code',how='left')
                finaldat.to_csv('data.csv',index=False,index_label=False)
                pd.Series(doneindex).to_csv('donelist.csv',index=False,index_label=False)
                print('done loop:', index)
        if len(doneindex) > 3:
            x=False
            break
    except:
        continue
df2.code.astype('str')

In [3]:
dlist = pd.DataFrame(list())
dlist.to_csv('donelist.csv')

In [5]:
k = 1
w = True
while w == True:
    try:
        k = k+1
        if k == 5:
            w = False
            print('done')
    except:
        break

done
